In [1]:
%config IPCompleter.greedy=True  # Podpowiedzi składni
file = open('kroA100.tsp.txt')
file2 = open('kroB100.tsp.txt')

In [2]:
next_line = file.readline()[0:-1]
while next_line[0] != "1":
    next_line = file.readline()[0:-1]
print(next_line)

1 1380 939


In [3]:
base_tab = []
while next_line!= "EOF":
    base_tab.append(next_line.split(" "))
    next_line = file.readline()[0:-1]

In [4]:
for x in range(len(base_tab)):
    for y in range(len(base_tab[0])):
        base_tab[x][y] = int(base_tab[x][y])
print(len(base_tab))
print(len(base_tab[0]))

100
3


In [5]:
import math

def round2(number):
    if (number - math.trunc(number) >= 0.5):
        return math.trunc(number) + 1
    else:
        return math.trunc(number)

In [6]:
import numpy as np
length_tab = np.zeros((len(base_tab),len(base_tab)), dtype=int)

for x in range(len(length_tab)):
    for y in range(len(length_tab[0])):
        length_tab[x][y] = round2(math.sqrt((base_tab[x][1] - base_tab[y][1])**2 + (base_tab[x][2] - base_tab[y][2])**2))  # długość między punktem "x" i "y" w tablicy base_tab
#print(length_tab)

GREEDY CYCLE:

In [7]:
import random

def get_nearest(point):  # znajdowanie najbliższego wierzchołka dla zadanego
    length = int
    nearest = int
    if point != 0:
        length = length_tab[point][0]
        nearest = 0
    else:
        length = length_tab[point][1]
        nearest = 1
    for x in range(len(length_tab)):
        if length_tab[point][x] < length and point != x:
            length = length_tab[point][x]
            nearest = x
    return nearest

def distance(path):
    if path == [0]:
        return 0
    dist = 0
    for x in range(len(path) - 1):
        dist+=length_tab[path[x]][path[x+1]]
    dist+=length_tab[path[len(path)-1]][path[0]]
    return dist

def print2(path):  # ze względu na uproszczenie pracy w programie, wierzchołki są liczone od 0 - zaś w podanym pliku od 1
    print("["+str(path[0] + 1), end = '')
    for x in range(len(path)-1):
        print(", " + str(path[x + 1] + 1), end = '')
    print("]")  

In [8]:
scores = []
best_path = [0]


for iterator in range(100):
    greedy_cycle = [random.randint(0, len(length_tab) - 1)]  # losuj pierwszy punkt
    greedy_cycle.append(get_nearest(greedy_cycle[0]))  # znajdź najbliższy wierzchołek

    while len(greedy_cycle) < len(length_tab) / 2:  # tak długo jak brakuje wierzchołków szukaj kolejnego
        min_cost = max(length_tab[greedy_cycle[0]]) + max(length_tab[greedy_cycle[1]])
        previous_point_index = 0
        best_point = 0
        for new_point in range(len(length_tab)):  # dla każdego punktu
            if new_point not in greedy_cycle:
                for x in range(len(greedy_cycle)):  # dla każdej krawędzi
                    point1 = greedy_cycle[x]
                    point2 = greedy_cycle[0]
                    if x != len(greedy_cycle) - 1:
                        point2 = greedy_cycle[x + 1]

                    cost = length_tab[point1][new_point] + length_tab[point2][new_point] - length_tab[point1][point2]
                    if cost < min_cost:
                        min_cost = cost
                        previous_point_index = x
                        best_point = new_point

        greedy_cycle.insert(previous_point_index + 1, best_point)  # wstaw nowy punkt za miejscem previous point
        
        
    scores.append(distance(greedy_cycle))
    if distance(greedy_cycle) < distance(best_path) or distance(best_path) == 0:
        best_path = greedy_cycle

        
print("Średnia ścieżka: "+str(sum(scores) / len(scores)))
print("Najdłuższa ścieżka: "+str(max(scores)))
print("Najkrótsza ścieżka: "+str(min(scores)))
print("Najlepsza ścieżka: ")
print2(best_path)

Średnia ścieżka: 12572.54
Najdłuższa ścieżka: 13266
Najkrótsza ścieżka: 11095
Najlepsza ścieżka: 
[62, 35, 86, 27, 12, 20, 7, 9, 57, 87, 51, 25, 81, 61, 58, 67, 28, 93, 1, 8, 92, 75, 97, 19, 53, 16, 70, 22, 94, 88, 18, 24, 38, 84, 79, 90, 49, 6, 63, 47, 11, 17, 15, 32, 45, 91, 98, 23, 77, 60]


REGRET ALGORITHM:
Pierwsze trzy wierzchołki

In [9]:
scores = []
best_path = [0]

for iterator in range(100):
    regret = [random.randint(0, len(length_tab) - 1)]  # losuj pierwszy punkt
    regret.append(get_nearest(regret[0]))  # znajdź najbliższy wierzchołek

    # Trzeci wierzchołek - dodawany na podstawie kosztu dodania, ponieważ na tym etapie żal dla wszystkich wierzchołków identyczny
    min_cost = max(length_tab[regret[0]]) + max(length_tab[regret[1]])
    previous_point_index = 0
    best_point = 0
    for new_point in range(len(length_tab)):  # dla każdego punktu
        if new_point not in regret:
            for x in range(len(regret)):  # dla każdej krawędzi
                point1 = regret[x]
                point2 = regret[0]
                if x != len(regret) - 1:
                    point2 = regret[x + 1]

                cost = length_tab[point1][new_point] + length_tab[point2][new_point] - length_tab[point1][point2]
                if cost < min_cost:
                    min_cost = cost
                    previous_point_index = x
                    best_point = new_point

    regret.insert(previous_point_index + 1, best_point)  # wstaw nowy punkt za miejscem previous point


    # Pozostałe wierzchołki
    while len(regret) < len(length_tab) / 2:  # tak długo jak brakuje wierzchołków szukaj kolejnego

        points_results = []  # wyniki wszystkich wierzchołków
        for new_point in range(len(length_tab)):  # dla każdego punktu
            if new_point not in regret:
                p_results = []  # wyniki tego jednego wierzchołka
                for x in range(len(regret)):  # dla każdej krawędzi
                    point1 = regret[x]
                    point2 = regret[0]
                    if x != len(regret) - 1:
                        point2 = regret[x + 1]

                    cost = length_tab[point1][new_point] + length_tab[point2][new_point] - length_tab[point1][point2]
                    p_results.append(cost)
                points_results.append(p_results)
            else:
                points_results.append([0, 0])

        # Sortowanie wyników
        for x in points_results:  
            x.sort()

        # Znajdowanie największego żalu
        highest_regret = 0  
        best_point = 0
        for x in range(len(points_results)):
            if x not in regret and points_results[x][1]-points_results[x][0] > highest_regret:
                highest_regret = points_results[x][1]-points_results[x][0]
                best_point = x

        # Wstawienie wierzchołka o największym żalu do najlepszej lokalizacji
        min_cost = max(length_tab[regret[0]]) + max(length_tab[regret[1]])
        previous_point_index = 0

        for x in range(len(regret)):
            point1 = regret[x]
            point2 = regret[0]
            if x != len(regret) - 1:
                point2 = regret[x + 1]

            cost = length_tab[point1][best_point] + length_tab[point2][best_point] - length_tab[point1][point2]
            if cost < min_cost:
                min_cost = cost
                previous_point_index = x

        regret.insert(previous_point_index + 1, best_point)
        
        
    scores.append(distance(regret))
    if distance(regret) < distance(best_path) or distance(best_path) == 0:
        best_path = regret
    
    

print("Średnia ścieżka: "+str(sum(scores) / len(scores)))
print("Najdłuższa ścieżka: "+str(max(scores)))
print("Najkrótsza ścieżka: "+str(min(scores)))
print("Najlepsza ścieżka: ")
print2(best_path)

Średnia ścieżka: 15742.78
Najdłuższa ścieżka: 17171
Najkrótsza ścieżka: 15007
Najlepsza ścieżka: 
[88, 53, 66, 4, 97, 56, 80, 42, 8, 92, 63, 1, 67, 58, 61, 81, 69, 54, 2, 50, 82, 95, 76, 37, 96, 30, 48, 100, 41, 43, 29, 34, 83, 9, 12, 27, 35, 62, 60, 23, 91, 32, 17, 59, 99, 24, 18, 94, 22, 16]
